In [1]:
import time
import requests
import dotenv
from data import *

BV = "BV1k24y197KC"

BV = "BV1YQ4y187V5"

bili_key = dotenv.get_key(".env", "BILI_KEY")
header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0",
    "Accept": "application/json, text/plain, */*",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6,ko;q=0.5",
}




爬虫获取视频json

In [2]:
response = requests.get("https://bili.zhouql.vip/meta/" + BV, headers = header)
response.encoding = 'utf-8'
j = response.json()
j

{'code': 0,
 'message': '0',
 'ttl': 1,
 'data': {'bvid': 'BV1YQ4y187V5',
  'aid': 705815123,
  'videos': 1,
  'tid': 231,
  'tname': '计算机技术',
  'copyright': 1,
  'pic': 'http://i1.hdslb.com/bfs/archive/d46d0708325f04b5ffdd897c76188f534193df98.jpg',
  'title': '说说浏览器内核的代码架构',
  'pubdate': 1699628719,
  'ctime': 1699628719,
  'desc': '说说浏览器内核的代码架构，简单说一下浏览器的内核需要哪些技术：GTK，http模块，2D矢量图形库，OpenGL，js脚本语言的引擎，html解析，排版+渲染。',
  'desc_v2': [{'raw_text': '说说浏览器内核的代码架构，简单说一下浏览器的内核需要哪些技术：GTK，http模块，2D矢量图形库，OpenGL，js脚本语言的引擎，html解析，排版+渲染。',
    'type': 1,
    'biz_id': 0}],
  'state': 0,
  'duration': 3104,
  'rights': {'bp': 0,
   'elec': 0,
   'download': 1,
   'movie': 0,
   'pay': 0,
   'hd5': 0,
   'no_reprint': 1,
   'autoplay': 1,
   'ugc_pay': 0,
   'is_cooperation': 0,
   'ugc_pay_preview': 0,
   'no_background': 0,
   'clean_mode': 0,
   'is_stein_gate': 0,
   'is_360': 0,
   'no_share': 0,
   'arc_pay': 0,
   'free_watch': 0},
  'owner': {'mid': 3493276204141323,
   'name': '底层技术栈',
   'face

获取视频信息，但是不添加下载链接

In [3]:
parts = []
for each in j["data"]["pages"]:
    new_part = VideoPart(
        cid = each["cid"],
        page = each["page"],
        part = each["part"],
        duration = each["duration"],
        dimension = each["dimension"],
        first_frame_url = each["first_frame"],
        download_url = None
    )
    parts.append(new_part)

for each in parts:
    print(each.__dict__)

{'cid': 1328111519, 'page': 1, 'part': '说说浏览器内核的代码架构', 'duration': 3104, 'dimension': {'width': 1920, 'height': 1080, 'rotate': 0}, 'first_frame_url': 'http://i1.hdslb.com/bfs/storyff/n231110sa3ij76p60jpwe01nunfov4mc_firsti.jpg', 'download_url': None}


获取视频合集信息

In [4]:
video_set = VideoSet(
        aid = j["data"]["aid"],
        bvid = j["data"]["bvid"],
        title = j["data"]["title"],
        part_num = j["data"]["videos"],
        parts = parts,
        cover_url = j["data"]["pic"],
)
video_set.__dict__

{'aid': 705815123,
 'bvid': 'BV1YQ4y187V5',
 'title': '说说浏览器内核的代码架构',
 'desc': None,
 'part_num': 1,
 'parts': [<VideoClass.VideoPart at 0x1c09b7bd9d0>],
 'cover_url': 'http://i1.hdslb.com/bfs/archive/d46d0708325f04b5ffdd897c76188f534193df98.jpg'}

获取视频下载链接

In [5]:
for each in video_set.parts:
    response = requests.get("https://bili.zhouql.vip/download/" + str(video_set.aid) + "/" + str(each.cid), headers = header)
    response.encoding = 'utf-8'
    j = response.json()
    each.download_url = j["data"]["durl"][0]["url"]
    time.sleep(3)
    print(each.__dict__)

{'cid': 1328111519, 'page': 1, 'part': '说说浏览器内核的代码架构', 'duration': 3104, 'dimension': {'width': 1920, 'height': 1080, 'rotate': 0}, 'first_frame_url': 'http://i1.hdslb.com/bfs/storyff/n231110sa3ij76p60jpwe01nunfov4mc_firsti.jpg', 'download_url': 'https://cn-jsnt-ct-01-23.bilivideo.com/upgcxcode/19/15/1328111519/1328111519-1-16.mp4?e=ig8euxZM2rNcNbRVhwdVhwdlhWdVhwdVhoNvNC8BqJIzNbfq9rVEuxTEnE8L5F6VnEsSTx0vkX8fqJeYTj_lta53NCM=&uipk=5&nbs=1&deadline=1701524455&gen=playurlv2&os=bcache&oi=2043500563&trid=0000dc6a4e6d06e64fdea6bcdb457dfe6c10h&mid=0&platform=html5&upsig=182f041cc01016656a0c514e19d756e0&uparams=e,uipk,nbs,deadline,gen,os,oi,trid,mid,platform&cdnid=9931&bvc=vod&nettype=0&f=h_0_0&bw=11909&logo=80000000'}


下载视频

In [6]:
from VideoUtils import VideoUtils

VideoUtils.download_videoset(video_set)

True